In [11]:
import pandas as pd
import numpy as np
from tensorflow import keras
from BinanceCollector import BinanceCollector
from HyperBacktest import HyperBacktest
from BreakdownCalculator import BreakdownCalculator
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split

In [118]:
def reshape(x):
    return np.reshape( x,(x.shape[0],1,x.shape[1]) )

In [90]:
def get_trades(symbol):
    collector = BinanceCollector(symbol,"2022-01-01","2023-01-02","15m")
    collector.start_collect()
    collector.df.columns = ["Open Time","Open","High","Low","Close","Volume","Close Time",
             "Quote asset volumne","Total Trades","Buyer Base Volume",
             "Taker buy quote asset volume","Ignore"]
    df = collector.df
    breakdown = BinanceCollector(symbol,"2022-01-01","2023-01-02","1m")
    breakdown.start_collect()
    breakdown.df.columns = ["Open Time","Open","High","Low","Close","Volume","Close Time",
             "Quote asset volumne","Total Trades","Buyer Base Volume",
             "Taker buy quote asset volume","Ignore"]
    breakdown_df = breakdown.df
    calculator = BreakdownCalculator(breakdown_df)

    # Setup metric columns
    df["EMA_20"] = df["Close"].ewm(span=20,adjust=False).mean()
    df["EMA_50"] = df["Close"].ewm(span=50,adjust=False).mean()
    df["EMA_200"] = df["Close"].ewm(span=200,adjust=False).mean()
    df["Open Time"] = pd.to_datetime(df["Open Time"],unit="ms")
    df["Close Time"] = pd.to_datetime(df["Close Time"]+1,unit="ms")

    df["Buy"] = np.where( (df["EMA_20"] > df["EMA_50"]) & ( df['EMA_20'].shift(1) <= df["EMA_50"].shift(1) ), 1, 0)
    df["Sell"] = np.where( (df["EMA_20"] < df["EMA_50"]) & ( df['EMA_20'].shift(1) >= df["EMA_50"].shift(1) ), 1, 0 )
    df["Returns"] = 0
    df = df[["Open","High","Low","Close","EMA_20","EMA_50","EMA_200","Open Time","Buy","Sell","Returns"]]
    normalize_cols = ["Open","High","Low","Close","EMA_20","EMA_50","EMA_200"]

    backtest = HyperBacktest(4,-2,calculator,df)
    backtest.run()
    final = backtest.df
    final[ normalize_cols ] = (final[ normalize_cols ] - final[normalize_cols].min())/(final[normalize_cols].max() - final[normalize_cols].min())
    return final

In [92]:
symbols = ["ETHUSDT","SOLUSDT","UNIUSDT"]
dfs = []
for i in symbols:
    df = get_trades(i)
    dfs.append(df)

df = pd.concat(dfs)
df.reset_index(inplace=True)
main = df.copy()

Before Drop: 524
After Drop: 419
Before Drop: 577
After Drop: 500
Before Drop: 615
After Drop: 525


In [93]:
df = main.copy()
df = df[ df['Buy'] == 1 ]

In [110]:
x = np.array(df[["EMA_20","EMA_50","EMA_200","Open","High","Low","Close"]])
y = np.where( df["Returns"] > 3,1,0 )

In [111]:
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2)

In [112]:
x_train = np.reshape( x_train, (x_train.shape[0],1,x_train.shape[1]) )
x_test = np.reshape( x_test, (x_test.shape[0],1,x_test.shape[1]) )
# y_train = np.reshape( y_train,( y_train.shape[0],1,y_train.shape[1] ) )

In [140]:
model = keras.Sequential([
    keras.layers.Dense(8, input_shape=(1,7),activation="sigmoid"),
    # keras.layers.LSTM(4,return_sequences=True),
    keras.layers.Dense(8,activation="relu"),
    keras.layers.Dense(2,activation="softmax"),
    keras.layers.Softmax()
])

model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
model.fit(x_train,y_train,epochs=20)

Train on 584 samples
Epoch 1/20
584/584 [==============================] - 1s 2ms/sample - loss: 0.6276 - acc: 0.8162
Epoch 2/20
584/584 [==============================] - 0s 115us/sample - loss: 0.5946 - acc: 0.8201
Epoch 3/20
584/584 [==============================] - 0s 86us/sample - loss: 0.5707 - acc: 0.8162
Epoch 4/20
584/584 [==============================] - 0s 103us/sample - loss: 0.5528 - acc: 0.8162
Epoch 5/20
584/584 [==============================] - 0s 118us/sample - loss: 0.5402 - acc: 0.8149
Epoch 6/20
584/584 [==============================] - 0s 108us/sample - loss: 0.5304 - acc: 0.8162
Epoch 7/20
584/584 [==============================] - 0s 99us/sample - loss: 0.5237 - acc: 0.8175
Epoch 8/20
584/584 [==============================] - 0s 130us/sample - loss: 0.5188 - acc: 0.8175
Epoch 9/20
584/584 [==============================] - 0s 113us/sample - loss: 0.5150 - acc: 0.8162
Epoch 10/20
584/584 [==============================] - 0s 103us/sample - loss: 0.5122 - acc:

In [141]:
model.evaluate(x_test,y_test)

147/147 [==============================] - 1s 7ms/sample - loss: 0.5594 - acc: 0.7590


[0.5593675796677466, 0.75903076]

In [142]:
df[50:60]

,index,Open,High,Low,Close,EMA_20,EMA_50,EMA_200,Open Time,Buy,Sell,Returns,End,Gap
104,8430,0.859340,0.864620,0.858823,0.865021,0.851875,0.850838,0.848465,2022-03-30 17:45:00,1,0,-1,2022-03-31 14:00:00,1196.0
106,8600,0.814858,0.819976,0.816009,0.819412,0.812435,0.811762,0.830672,2022-04-01 12:45:00,1,0,4,2022-04-01 15:16:00,1239.0
109,8857,0.888383,0.888806,0.890120,0.889501,0.889453,0.889196,0.885040,2022-04-04 05:45:00,1,0,-2,2022-04-04 15:03:00,882.0
111,9154,0.787625,0.788442,0.788690,0.789642,0.788940,0.788812,0.824621,2022-04-07 08:45:00,1,0,-1,2022-04-07 15:32:00,2558.0
114,9273,0.804200,0.816552,0.804634,0.813200,0.804760,0.804379,0.809691,2022-04-08 14:45:00,1,0,-1,2022-04-08 17:35:00,926.0
116,9451,0.796234,0.799672,0.798665,0.799324,0.799295,0.799102,0.800402,2022-04-10 11:45:00,1,0,-1,2022-04-11 00:09:00,2530.0
118,9626,0.718783,0.720729,0.718695,0.721084,0.716026,0.715564,0.748466,2022-04-12 08:00:00,1,0,-1,2022-04-12 19:59:00,1911.0
120,9696,0.718381,0.719531,0.718826,0.720712,0.719686,0.719339,0.734891,2022-04-13 01:45:00,1,0,3,2022-04-14 01:44:00,346.0
125,10106,0.726993,0.726159,0.728011,0.726437,0.727897,0.727754,0.728047,2022-04-17 09:15:00,1,0,-2,2022-04-17 23:41:00,665.0
126,10236,0.704467,0.707265,0.705308,0.704538,0.692796,0.691967,0.709142,2022-04-18 18:00:00,1,0,3,2022-04-19 14:00:00,1117.0


In [143]:
test_x = df[50:60][["EMA_20","EMA_50","EMA_200","Open","High","Low","Close"]]
test_x = reshape(np.array(test_x))

In [144]:
model.predict(test_x)

array([[[0.72666633, 0.27333364]],

       [[0.72651565, 0.27348438]],

       [[0.7267697 , 0.27323034]],

       [[0.72641486, 0.27358514]],

       [[0.72648126, 0.2735187 ]],

       [[0.7264269 , 0.2735731 ]],

       [[0.7260848 , 0.2739152 ]],

       [[0.7260719 , 0.2739281 ]],

       [[0.7260891 , 0.27391085]],

       [[0.7259681 , 0.27403188]]], dtype=float32)